#Media pipe

In [ ]:
import numpy as np
np.arr  # Press Tab after "arr"


In [5]:
import cv2
import mediapipe as mp

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Start capturing video
cap = cv2.VideoCapture(0) # Use 0 for webcam, or provide a video file path



while cap.isOpened():
    ret ,frame = cap.read()
    if not ret :
        break

    # Convert frame to RGB (MediaPipe requires RGB format)
    rgb_frame = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)

    # Get pose landmarks
    results = pose.process(rgb_frame)

    # Draw pose landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)

    # Show output
    cv2.imshow("Pose Datection",frame)

    # Press 'q' to exit
    if cv2.waitKey(1) & 0xff == ord('q'):
        break

cap.release()
cap.destroyAllWindows()

AttributeError: 'cv2.VideoCapture' object has no attribute 'destroyAllWindows'

# MediaPipe detects 33 keypoints (landmarks) on the body.
Some important ones:

    Nose: results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE]
    Shoulders: LEFT_SHOULDER, RIGHT_SHOULDER
    Elbows: LEFT_ELBOW, RIGHT_ELBOW
    Knees: LEFT_KNEE, RIGHT_KNEE

Full list: MediaPipe Pose Landmarks

In [1]:
%config Completer.use_jedi = False


🛠️ Step 4: Print Shoulder Coordinates & Save Video

Now, let’s modify the script to:

✅ Print left & right shoulder coordinates (x, y).

✅ Save the pose-detected video using cv2.VideoWriter().

In [ ]:
import cv2 
import mediapipe as mp

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Start capturing video
cap = cv2.VideoCapture(0) # Use 0 for webcam, or provide a video file path

# Get video width, height, and FPS for saving
frame_width = int(cap.get(3))
frame_height = int(cap.get(4))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define video writer to save the output
out = cv2.VideoWriter('pose_output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))

while cap.isOpened() :
    
    ret,frame = cap.read()

    if not ret:
        break

    # Convert frame to RGB (MediaPipe requires RGB format)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)    

    #get Pose landmarks
    results = pose.process(rgb_frame)


    # Draw pose Landmarks
    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)

        #get shoulder coordinates
        left_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.LEFT_SHOULDER]
        right_shoulder = results.pose_landmarks.landmark[mp_pose.PoseLandmark.RIGHT_SHOULDER]

        # Convert normalized coordinates (0-1) to pixel values
        left_shoulder_x,left_shoulder_y = int(left_shoulder.x * frame_width) , int(left_shoulder.y * frame_height)
        right_shoulder_x,right_shoulder_y = int(right_shoulder.x * frame_width) , int(right_shoulder.y * frame_height)
        
        # print coordinates
        print(f"Left Shoulder: ({left_shoulder_x}, {left_shoulder_y}), Right Shoulder: ({right_shoulder_x}, {right_shoulder_y})")

        # Draw Circles on Shoulders
        cv2.circle(frame,(left_shoulder_x,left_shoulder_y), 5,(0,255,0),-1)
        cv2.circle(frame,(right_shoulder_x,right_shoulder_y), 5,(0,255,0),-1)

    # Save video Frame
    out.write(frame)

    #show output
    cv2.imshow("Pose Detection",frame)


    #press q tp quit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
out.release()
cv2.destroyAllWindows()



Left Shoulder: (523, 391), Right Shoulder: (214, 388)
Left Shoulder: (525, 394), Right Shoulder: (211, 399)
Left Shoulder: (525, 395), Right Shoulder: (210, 401)
Left Shoulder: (525, 396), Right Shoulder: (209, 402)
Left Shoulder: (526, 396), Right Shoulder: (208, 403)
Left Shoulder: (526, 397), Right Shoulder: (208, 403)
Left Shoulder: (526, 398), Right Shoulder: (208, 404)
Left Shoulder: (526, 398), Right Shoulder: (208, 403)
Left Shoulder: (526, 398), Right Shoulder: (208, 403)
Left Shoulder: (526, 398), Right Shoulder: (208, 403)
Left Shoulder: (527, 398), Right Shoulder: (208, 403)
Left Shoulder: (527, 398), Right Shoulder: (208, 403)
Left Shoulder: (527, 398), Right Shoulder: (208, 403)
Left Shoulder: (527, 398), Right Shoulder: (207, 403)
Left Shoulder: (528, 399), Right Shoulder: (208, 403)
Left Shoulder: (529, 399), Right Shoulder: (208, 403)
Left Shoulder: (530, 399), Right Shoulder: (210, 403)
Left Shoulder: (530, 398), Right Shoulder: (211, 403)
Left Shoulder: (532, 397), R

KeyboardInterrupt: 

## 📌 Step 5: Measure Angle Between Joints (Shoulder, Elbow, Wrist)

#### Now that we can track the shoulders, let’s calculate the angle between joints to evaluate form. This is useful for checking exercise posture like bicep curls, squats, or push-ups.
## 🔢 How to Calculate the Angle?

We use the cosine rule:
θ=cos⁡−1(b2+c2−a22bc)
θ=cos−1(2bcb2+c2−a2​)

where:

    a, b, c are the distances between points.
    θ is the joint angle (in degrees).

We'll calculate elbow flexion angle using:

    Shoulder
    Elbow
    Wrist

In [ ]:
import cv2 
import mediapipe as mp
import numpy as np

# Initialize MediaPipe Pose model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

#start capturing video
cap = cv2.VideoCapture(0)

def calculate_angles(a,b,c):

    a = np.array(a) #first point
    b = np.array(b) # mid Point (joint)
    c = np.array(c) # last point

    # Calculate vectors
    ba = a - b
    bc = c-b

    # Compute cosine of the angle
    cosine_angle = np.dot(ba,bc) / ( np.linalg.norm(ba) * np.linalg.norm(bc))

    #convert to degrees
    angle = np.degrees(np.arccos(cosine_angle))

    return angle

while cap.isOpened() :
    ret ,frame = cap.read()
    
    if not ret :
        break

    frame_height,frame_width , _ = frame.shape

    #convert to rgb
    rgb_frame = cv2.cvtColor(frame , cv2.COLOR_BGR2RGB)

    results = pose.process(rgb_frame)

    if results.pose_landmarks:
        mp_drawing.draw_landmarks(frame,results.pose_landmarks,mp_pose.POSE_CONNECTIONS)

    landmarks = results.pose_landmarks.landmark

    shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * frame_width),
                int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * frame_height))

    elbow = (int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x * frame_width),
                int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y * frame_height))

    wrist = (int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x * frame_width),
                int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y * frame_height))

    angle = calculate_angles(shoulder , elbow , wrist)


    cv2.putText(frame, f"Angle: {int(angle)} deg", (elbow[0] - 50, elbow[1] - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)


    cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
    cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
    cv2.circle(frame, wrist, 5, (0, 0, 255), -1)


    cv2.imshow("Pose Detection with Angle calculation" , frame)


    if cv2.waitKey(1) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()








AttributeError: 'NoneType' object has no attribute 'landmark'

: 

## 🔥 Next Steps: Adding Rep Counting!

#### Now that we detect arm angles, we can count reps for an exercise like bicep curls using a simple threshold-based approach.
### 🛠 Plan for Rep Counting:

#### 1️⃣ Detect Curl Motion: Track elbow angle
#### 2️⃣ Set Thresholds:

    Top Position: Arm is straight (angle ~160°-180°)
    Bottom Position: Arm is bent (angle ~30°-60°)
    3️⃣ Count Reps:
    A rep is counted when the arm moves from top to bottom and back up
    Use a state machine to avoid counting multiple times
    4️⃣ Show Reps on Screen 🎯

In [ ]:
pip install streamlit


^C


  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached rich-13.9.4-py3-none-any.whl.metadata (18 kB)
  Using cached tenacity-9.0.0-py3-none-any.whl.metadata (1.2 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached jinja2-3.1.5-py3-none-any.whl.metadata (2.6 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached markdown_it_py-3.0.

In [ ]:
!pip install plotly

^C


   ---------------------------------------- 0.0/14.8 MB ? eta -:--:--
   -------- ------------------------------- 3.1/14.8 MB 23.1 MB/s eta 0:00:01
   ------------ --------------------------- 4.5/14.8 MB 12.8 MB/s eta 0:00:01
   --------------------- ------------------ 7.9/14.8 MB 15.2 MB/s eta 0:00:01
   ------------------------ --------------- 9.2/14.8 MB 11.9 MB/s eta 0:00:01
   ----------------------------- ---------- 10.7/14.8 MB 10.8 MB/s eta 0:00:01
   ---------------------------------------  14.7/14.8 MB 12.1 MB/s eta 0:00:01
   ---------------------------------------- 14.8/14.8 MB 11.8 MB/s eta 0:00:00


In [3]:
import streamlit as st
import cv2
import mediapipe as mp
import numpy as np
import tempfile
import plotly.graph_objects as go

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# Function for counting reps
def count_reps(angles, threshold_down=160, threshold_up=50):
    count = 0
    direction = 0  # 0: neutral, 1: going down, 2: going up
    
    for angle in angles:
        if angle > threshold_down:
            if direction == 2:
                count += 1
                direction = 0
        elif angle < threshold_up:
            direction = 2
        else:
            direction = 1
    
    return count

# Function for Pose Detection & Rep Counting
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    
    out = cv2.VideoWriter('output.mp4', cv2.VideoWriter_fourcc(*'mp4v'), fps, (frame_width, frame_height))
    angles_list = []
    skeleton_points = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        if results.pose_landmarks:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            landmarks = results.pose_landmarks.landmark
            
            shoulder = (int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x * frame_width),
                        int(landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y * frame_height))
            elbow = (int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].x * frame_width),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_ELBOW].y * frame_height))
            wrist = (int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST].x * frame_width),
                     int(landmarks[mp_pose.PoseLandmark.LEFT_WRIST].y * frame_height))
            
            angle = calculate_angle(shoulder, elbow, wrist)
            angles_list.append(angle)
            
            skeleton_points.append([
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].x, 
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].y, 
                landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER].z
            ])
            
            cv2.putText(frame, f'Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
            
            cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
            cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
            cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        
        out.write(frame)
    
    cap.release()
    out.release()
    
    reps = count_reps(angles_list)
    return 'output.mp4', reps, skeleton_points

# Function to generate 3D visualization
def generate_3d_pose(skeleton_points):
    fig = go.Figure()
    x, y, z = zip(*skeleton_points)
    
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=5, color='blue')))
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='3D Pose Visualization')
    return fig

# Streamlit UI
st.title("AI Fitness Coach - Pose Estimation, Rep Counter & 3D Visualization")

uploaded_file = st.file_uploader("Upload an exercise video", type=["mp4", "mov", "avi"])
view_option = st.radio("Select View Mode:", ["Skeleton (Lines)", "3D Muscle Body"])

if uploaded_file is not None:
    with tempfile.NamedTemporaryFile(delete=False) as temp_file:
        temp_file.write(uploaded_file.read())
        video_path = temp_file.name
    
    st.video(video_path)
    st.write("Processing video...")
    
    output_video_path, rep_count, skeleton_points = process_video(video_path)
    
    st.video(output_video_path)
    st.success(f"Processing Complete! Total Reps Counted: {rep_count}")
    
    if view_option == "Skeleton (Lines)":
        st.plotly_chart(generate_3d_pose(skeleton_points))
    else:
        st.write("3D Muscle Body Visualization (Coming Soon)")


2025-03-01 23:39:34.688 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 23:39:35.267 
  command:

    streamlit run c:\Users\dewan\Coding\Python\fito\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-03-01 23:39:35.268 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 23:39:35.268 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 23:39:35.268 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 23:39:35.268 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 23:39:35.268 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-03-01 23:39:35.268 Thread 'MainTh

In [5]:
import cv2
import mediapipe as mp
import numpy as np
import plotly.graph_objects as go

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# Function for counting reps
def count_reps(angles, threshold_down=160, threshold_up=50):
    count = 0
    direction = 0  # 0: neutral, 1: going down, 2: going up
    
    for angle in angles:
        if angle > threshold_down:
            if direction == 2:
                count += 1
                direction = 0
        elif angle < threshold_up:
            direction = 2
        else:
            direction = 1
    
    return count

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    angles_list = []
    skeleton_points = []
    peak_concentration = 0
    
    cv2.namedWindow("Live Pose Tracking", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Live Pose Tracking", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                landmarks = results.pose_landmarks.landmark
                
                joints = {
                    'left_shoulder': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                    'right_shoulder': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
                }
                
                for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                    shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                                int(landmarks[shoulder_idx].y * frame_height))
                    elbow = (int(landmarks[elbow_idx].x * frame_width),
                             int(landmarks[elbow_idx].y * frame_height))
                    wrist = (int(landmarks[wrist_idx].x * frame_width),
                             int(landmarks[wrist_idx].y * frame_height))
                    
                    angle = calculate_angle(shoulder, elbow, wrist)
                    angles_list.append(angle)
                    
                    skeleton_points.append([
                        landmarks[shoulder_idx].x, 
                        landmarks[shoulder_idx].y, 
                        landmarks[shoulder_idx].z
                    ])
                    
                    cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                    cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                    cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except Exception as e:
            print(f"Error processing landmarks: {e}")
        
        # Display muscle concentration bar
        muscle_concentration = angles_list[-1] if angles_list else 0  # Use the latest angle for faster response
        peak_concentration = min(peak_concentration, muscle_concentration)
        bar_length = int(muscle_concentration / 180 * 200)  # Scale for visualization
        cv2.rectangle(frame, (50, 50), (50 + bar_length, 70), (0, 255, 0), -1)
        cv2.putText(frame, f'Muscle Concentration: {int(muscle_concentration)}%', (50, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # Display peak concentration
        cv2.putText(frame, f'Peak Concentration: {int(peak_concentration)}%', (50, 90), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    reps = count_reps(angles_list)
    return reps, skeleton_points

# Function to generate 3D visualization
def generate_3d_pose(skeleton_points):
    fig = go.Figure()
    x, y, z = zip(*skeleton_points)
    
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=5, color='blue')))
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='3D Pose Visualization')
    return fig

if __name__ == "__main__":
    live_tracking()


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import plotly.graph_objects as go

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

# Function for counting reps
def count_reps(angles, threshold_down=160, threshold_up=50):
    count = 0
    direction = 0  # 0: neutral, 1: going down, 2: going up
    
    for angle in angles:
        if angle > threshold_down:
            if direction == 2:
                count += 1
                direction = 0
        elif angle < threshold_up:
            direction = 2
        else:
            direction = 1
    
    return count

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    left_angles = []
    right_angles = []
    skeleton_points = []
    left_peak_concentration = 0
    right_peak_concentration = 0
    
    cv2.namedWindow("Live Pose Tracking", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Live Pose Tracking", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                landmarks = results.pose_landmarks.landmark
                
                joints = {
                    'left': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                    'right': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
                }
                
                for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                    shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                                int(landmarks[shoulder_idx].y * frame_height))
                    elbow = (int(landmarks[elbow_idx].x * frame_width),
                             int(landmarks[elbow_idx].y * frame_height))
                    wrist = (int(landmarks[wrist_idx].x * frame_width),
                             int(landmarks[wrist_idx].y * frame_height))
                    
                    angle = calculate_angle(shoulder, elbow, wrist)
                    if side == 'left':
                        left_angles.append(angle)
                    else:
                        right_angles.append(angle)
                    
                    skeleton_points.append([
                        landmarks[shoulder_idx].x, 
                        landmarks[shoulder_idx].y, 
                        landmarks[shoulder_idx].z
                    ])
                    
                    cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                    cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                    cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except Exception as e:
            print(f"Error processing landmarks: {e}")
        
        # Display muscle concentration bars
        left_concentration = left_angles[-1] if left_angles else 0
        right_concentration = right_angles[-1] if right_angles else 0
        
        left_peak_concentration = max(left_peak_concentration, left_concentration)
        right_peak_concentration = max(right_peak_concentration, right_concentration)
        
        left_bar_length = int(left_concentration / 180 * 200)
        right_bar_length = int(right_concentration / 180 * 200)
        
        cv2.rectangle(frame, (50, 50), (50 + left_bar_length, 70), (0, 255, 0), -1)
        cv2.putText(frame, f'Left Concentration: {int(left_concentration)}%', (50, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.rectangle(frame, (50, 100), (50 + right_bar_length, 120), (255, 0, 0), -1)
        cv2.putText(frame, f'Right Concentration: {int(right_concentration)}%', (50, 95), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # Display peak concentrations
        cv2.putText(frame, f'Left Peak: {int(left_peak_concentration)}%', (50, 140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        cv2.putText(frame, f'Right Peak: {int(right_peak_concentration)}%', (50, 160), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    left_reps = count_reps(left_angles)
    right_reps = count_reps(right_angles)
    return left_reps, right_reps, skeleton_points

# Function to generate 3D visualization
def generate_3d_pose(skeleton_points):
    fig = go.Figure()
    x, y, z = zip(*skeleton_points)
    
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=5, color='blue')))
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='3D Pose Visualization')
    return fig

if __name__ == "__main__":
    live_tracking()


Error processing landmarks: cannot convert float NaN to integer


C:\Users\dewan\AppData\Local\Temp\ipykernel_22700\3726280665.py:21: RuntimeWarning: invalid value encountered in arccos
  angle = np.degrees(np.arccos(cosine_angle))


ValueError: cannot convert float NaN to integer

: 

In [1]:
import cv2
import mediapipe as mp
import numpy as np
import plotly.graph_objects as go

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))  # Ensure valid range
    return angle

# Function for counting reps
def count_reps(angles, threshold_down=160, threshold_up=50):
    count = 0
    direction = 0  # 0: neutral, 1: going down, 2: going up
    
    for angle in angles:
        if angle > threshold_down:
            if direction == 2:
                count += 1
                direction = 0
        elif angle < threshold_up:
            direction = 2
        else:
            direction = 1
    
    return count

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    left_angles = []
    right_angles = []
    skeleton_points = []
    left_peak_concentration = 0
    right_peak_concentration = 0
    
    cv2.namedWindow("Live Pose Tracking", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Live Pose Tracking", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            if results.pose_landmarks:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                landmarks = results.pose_landmarks.landmark
                
                joints = {
                    'left': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                    'right': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
                }
                
                for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                    shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                                int(landmarks[shoulder_idx].y * frame_height))
                    elbow = (int(landmarks[elbow_idx].x * frame_width),
                             int(landmarks[elbow_idx].y * frame_height))
                    wrist = (int(landmarks[wrist_idx].x * frame_width),
                             int(landmarks[wrist_idx].y * frame_height))
                    
                    angle = calculate_angle(shoulder, elbow, wrist)
                    angle = min(angle, 180)  # Normalize angle range
                    
                    if side == 'left':
                        left_angles.append(angle)
                    else:
                        right_angles.append(angle)
                    
                    skeleton_points.append([
                        landmarks[shoulder_idx].x, 
                        landmarks[shoulder_idx].y, 
                        landmarks[shoulder_idx].z
                    ])
                    
                    cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                    cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                    cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except Exception as e:
            print(f"Error processing landmarks: {e}")
        
        # Display muscle concentration bars
        left_concentration = left_angles[-1] if left_angles else 0
        right_concentration = right_angles[-1] if right_angles else 0
        
        left_peak_concentration = max(left_peak_concentration, left_concentration)
        right_peak_concentration = max(right_peak_concentration, right_concentration)
        
        left_bar_length = int(left_concentration / 180 * 200)
        right_bar_length = int(right_concentration / 180 * 200)
        
        cv2.rectangle(frame, (50, 50), (50 + left_bar_length, 70), (0, 255, 0), -1)
        cv2.putText(frame, f'Left Concentration: {int(left_concentration)}%', (50, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.rectangle(frame, (50, 100), (50 + right_bar_length, 120), (255, 0, 0), -1)
        cv2.putText(frame, f'Right Concentration: {int(right_concentration)}%', (50, 95), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        # Display peak concentrations
        cv2.putText(frame, f'Left Peak: {int(left_peak_concentration)}%', (50, 140), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        cv2.putText(frame, f'Right Peak: {int(right_peak_concentration)}%', (50, 160), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 2)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    
    left_reps = count_reps(left_angles)
    right_reps = count_reps(right_angles)
    return left_reps, right_reps, skeleton_points

# Function to generate 3D visualization
def generate_3d_pose(skeleton_points):
    fig = go.Figure()
    x, y, z = zip(*skeleton_points)
    
    fig.add_trace(go.Scatter3d(x=x, y=y, z=z, mode='markers+lines', marker=dict(size=5, color='blue')))
    fig.update_layout(scene=dict(xaxis_title='X', yaxis_title='Y', zaxis_title='Z'), title='3D Pose Visualization')
    return fig

if __name__ == "__main__":
    live_tracking()


In [2]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))  # Ensure valid range
    return angle

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    left_angles = []
    right_angles = []
    left_peak_concentration = 0
    right_peak_concentration = 0
    
    cv2.namedWindow("Live Pose Tracking", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Live Pose Tracking", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            results.pose_landmarks.landmark  # Attempt to access landmarks
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            landmarks = results.pose_landmarks.landmark
            
            joints = {
                'left': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                'right': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
            }
            
            for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                            int(landmarks[shoulder_idx].y * frame_height))
                elbow = (int(landmarks[elbow_idx].x * frame_width),
                         int(landmarks[elbow_idx].y * frame_height))
                wrist = (int(landmarks[wrist_idx].x * frame_width),
                         int(landmarks[wrist_idx].y * frame_height))
                
                angle = calculate_angle(shoulder, elbow, wrist)
                angle = min(angle, 180)  # Normalize angle range
                
                if side == 'left':
                    left_angles.append(angle)
                else:
                    right_angles.append(angle)
                
                cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except:
            pass
        
        # Display muscle concentration bars based on peak at 30-degree angle
        left_concentration = max(0, 100 - abs(left_angles[-1] - 30)) if left_angles else 0
        right_concentration = max(0, 100 - abs(right_angles[-1] - 30)) if right_angles else 0
        
        left_peak_concentration = max(left_peak_concentration, left_concentration)
        right_peak_concentration = max(right_peak_concentration, right_concentration)
        
        left_bar_length = int(left_concentration)
        right_bar_length = int(right_concentration)
        
        cv2.rectangle(frame, (50, 50), (50 + left_bar_length * 2, 70), (0, 255, 0), -1)
        cv2.putText(frame, f'Left Concentration: {int(left_concentration)}%', (50, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.rectangle(frame, (50, 100), (50 + right_bar_length * 2, 120), (255, 0, 0), -1)
        cv2.putText(frame, f'Right Concentration: {int(right_concentration)}%', (50, 95), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    live_tracking()


In [3]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))  # Ensure valid range
    return angle

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    left_angles = []
    right_angles = []
    left_peak_concentration = 0
    right_peak_concentration = 0
    
    cv2.namedWindow("Live Pose Tracking", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Live Pose Tracking", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        if results.pose_landmarks and results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility > 0.6:
            mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
            
            landmarks = results.pose_landmarks.landmark
            
            joints = {
                'left': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                'right': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
            }
            
            for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                            int(landmarks[shoulder_idx].y * frame_height))
                elbow = (int(landmarks[elbow_idx].x * frame_width),
                         int(landmarks[elbow_idx].y * frame_height))
                wrist = (int(landmarks[wrist_idx].x * frame_width),
                         int(landmarks[wrist_idx].y * frame_height))
                
                angle = calculate_angle(shoulder, elbow, wrist)
                angle = min(angle, 180)  # Normalize angle range
                
                if side == 'left':
                    left_angles.append(angle)
                else:
                    right_angles.append(angle)
                
                cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                
                cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        
        # Display muscle concentration bars based on peak at 30-degree angle
        left_concentration = max(0, 100 - abs(left_angles[-1] - 30)) if left_angles else 0
        right_concentration = max(0, 100 - abs(right_angles[-1] - 30)) if right_angles else 0
        
        left_peak_concentration = max(left_peak_concentration, left_concentration)
        right_peak_concentration = max(right_peak_concentration, right_concentration)
        
        left_bar_length = int(left_concentration)
        right_bar_length = int(right_concentration)
        
        cv2.rectangle(frame, (50, 50), (50 + left_bar_length * 2, 70), (0, 255, 0), -1)
        cv2.putText(frame, f'Left Concentration: {int(left_concentration)}%', (50, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.rectangle(frame, (50, 100), (50 + right_bar_length * 2, 120), (255, 0, 0), -1)
        cv2.putText(frame, f'Right Concentration: {int(right_concentration)}%', (50, 95), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    live_tracking()


In [4]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))  # Ensure valid range
    return angle

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    left_angles = []
    right_angles = []
    left_peak_concentration = 0
    right_peak_concentration = 0
    
    cv2.namedWindow("Live Pose Tracking", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Live Pose Tracking", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            if results.pose_landmarks and results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility > 0.6:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                landmarks = results.pose_landmarks.landmark
                
                joints = {
                    'left': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                    'right': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
                }
                
                for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                    shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                                int(landmarks[shoulder_idx].y * frame_height))
                    elbow = (int(landmarks[elbow_idx].x * frame_width),
                             int(landmarks[elbow_idx].y * frame_height))
                    wrist = (int(landmarks[wrist_idx].x * frame_width),
                             int(landmarks[wrist_idx].y * frame_height))
                    
                    angle = calculate_angle(shoulder, elbow, wrist)
                    angle = min(angle, 180)  # Normalize angle range
                    
                    if side == 'left':
                        left_angles.append(angle)
                    else:
                        right_angles.append(angle)
                    
                    cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                    
                    cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                    cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                    cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except Exception as e:
            pass
        
        # Display muscle concentration bars based on peak at 30-degree angle
        left_concentration = max(0, 100 - abs(left_angles[-1] - 30)) if left_angles else 0
        right_concentration = max(0, 100 - abs(right_angles[-1] - 30)) if right_angles else 0
        
        left_peak_concentration = max(left_peak_concentration, left_concentration)
        right_peak_concentration = max(right_peak_concentration, right_concentration)
        
        left_bar_length = int(left_concentration)
        right_bar_length = int(right_concentration)
        
        cv2.rectangle(frame, (50, 50), (50 + left_bar_length * 2, 70), (0, 255, 0), -1)
        cv2.putText(frame, f'Left Concentration: {int(left_concentration)}%', (50, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.rectangle(frame, (50, 100), (50 + right_bar_length * 2, 120), (255, 0, 0), -1)
        cv2.putText(frame, f'Right Concentration: {int(right_concentration)}%', (50, 95), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    live_tracking()


C:\Users\dewan\AppData\Local\Temp\ipykernel_22168\1964957081.py:19: RuntimeWarning: invalid value encountered in scalar divide
  cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))


In [1]:
import cv2
import mediapipe as mp
import numpy as np

# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))  # Ensure valid range
    return angle

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    left_angles = []
    right_angles = []
    left_peak_concentration = 0
    right_peak_concentration = 0
    
    cv2.namedWindow("Live Pose Tracking", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Live Pose Tracking", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            if results.pose_landmarks and results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility > 0.6:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                landmarks = results.pose_landmarks.landmark
                
                joints = {
                    'left': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                    'right': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
                }
                
                for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                    if (landmarks[shoulder_idx].visibility > 0.6 and 
                        landmarks[elbow_idx].visibility > 0.6 and 
                        landmarks[wrist_idx].visibility > 0.6):
                        
                        shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                                    int(landmarks[shoulder_idx].y * frame_height))
                        elbow = (int(landmarks[elbow_idx].x * frame_width),
                                 int(landmarks[elbow_idx].y * frame_height))
                        wrist = (int(landmarks[wrist_idx].x * frame_width),
                                 int(landmarks[wrist_idx].y * frame_height))
                        
                        angle = calculate_angle(shoulder, elbow, wrist)
                        angle = min(angle, 180)  # Normalize angle range
                        
                        if side == 'left':
                            left_angles.append(angle)
                        else:
                            right_angles.append(angle)
                        
                        cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        
                        cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                        cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                        cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except Exception as e:
            pass
        
        # Display muscle concentration bars based on peak at 30-degree angle
        left_concentration = max(0, 100 - abs(left_angles[-1] - 30)) if left_angles else 0
        right_concentration = max(0, 100 - abs(right_angles[-1] - 30)) if right_angles else 0
        
        left_peak_concentration = max(left_peak_concentration, left_concentration)
        right_peak_concentration = max(right_peak_concentration, right_concentration)
        
        left_bar_length = int(left_concentration)
        right_bar_length = int(right_concentration)
        
        cv2.rectangle(frame, (50, 50), (50 + left_bar_length * 2, 70), (0, 255, 0), -1)
        cv2.putText(frame, f'Left Concentration: {int(left_concentration)}%', (50, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.rectangle(frame, (50, 100), (50 + right_bar_length * 2, 120), (255, 0, 0), -1)
        cv2.putText(frame, f'Right Concentration: {int(right_concentration)}%', (50, 95), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    live_tracking()


In [2]:
import os
import cv2
import mediapipe as mp
import numpy as np

# Optimize TensorFlow threading (if used elsewhere in the project)
os.environ["OMP_NUM_THREADS"] = "8"  # OpenMP threads
os.environ["TF_NUM_INTEROP_THREADS"] = "8"  
os.environ["TF_NUM_INTRAOP_THREADS"] = "8"  

# Enable OpenCV multi-threading
cv2.setNumThreads(8)  # Adjust based on your CPU cores

# Improve NumPy operations
np.seterr(over='ignore')  # Prevent floating-point warnings


# Initialize Pose Model
mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
pose = mp_pose.Pose()

# Function to calculate angles
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    
    ba = a - b
    bc = c - b
    
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))  # Ensure valid range
    return angle

# Function for Real-time Pose Detection & Rep Counting
def live_tracking():
    cap = cv2.VideoCapture(0)
    left_angles = []
    right_angles = []
    left_peak_concentration = 0
    right_peak_concentration = 0
    
    cv2.namedWindow("Live Pose Tracking", cv2.WINDOW_NORMAL)
    cv2.setWindowProperty("Live Pose Tracking", cv2.WND_PROP_FULLSCREEN, cv2.WINDOW_FULLSCREEN)
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        frame_width = int(cap.get(3))
        frame_height = int(cap.get(4))
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = pose.process(rgb_frame)
        
        try:
            if results.pose_landmarks and results.pose_landmarks.landmark[mp_pose.PoseLandmark.NOSE].visibility > 0.6:
                mp_drawing.draw_landmarks(frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
                
                landmarks = results.pose_landmarks.landmark
                
                joints = {
                    'left': (mp_pose.PoseLandmark.LEFT_SHOULDER, mp_pose.PoseLandmark.LEFT_ELBOW, mp_pose.PoseLandmark.LEFT_WRIST),
                    'right': (mp_pose.PoseLandmark.RIGHT_SHOULDER, mp_pose.PoseLandmark.RIGHT_ELBOW, mp_pose.PoseLandmark.RIGHT_WRIST)
                }
                
                for side, (shoulder_idx, elbow_idx, wrist_idx) in joints.items():
                    if (landmarks[shoulder_idx].visibility > 0.6 and 
                        landmarks[elbow_idx].visibility > 0.6 and 
                        landmarks[wrist_idx].visibility > 0.6):
                        
                        shoulder = (int(landmarks[shoulder_idx].x * frame_width),
                                    int(landmarks[shoulder_idx].y * frame_height))
                        elbow = (int(landmarks[elbow_idx].x * frame_width),
                                 int(landmarks[elbow_idx].y * frame_height))
                        wrist = (int(landmarks[wrist_idx].x * frame_width),
                                 int(landmarks[wrist_idx].y * frame_height))
                        
                        angle = calculate_angle(shoulder, elbow, wrist)
                        angle = min(angle, 180)  # Normalize angle range
                        
                        if side == 'left':
                            left_angles.append(angle)
                        else:
                            right_angles.append(angle)
                        
                        cv2.putText(frame, f'{side.capitalize()} Angle: {int(angle)}', (elbow[0] - 50, elbow[1] - 20), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                        
                        cv2.circle(frame, shoulder, 5, (255, 0, 0), -1)
                        cv2.circle(frame, elbow, 5, (0, 255, 0), -1)
                        cv2.circle(frame, wrist, 5, (0, 0, 255), -1)
        except Exception as e:
            pass
        
        # Display muscle concentration bars based on peak at 30-degree angle
        left_concentration = max(0, 100 - abs(left_angles[-1] - 30)) if left_angles else 0
        right_concentration = max(0, 100 - abs(right_angles[-1] - 30)) if right_angles else 0
        
        left_peak_concentration = max(left_peak_concentration, left_concentration)
        right_peak_concentration = max(right_peak_concentration, right_concentration)
        
        left_bar_length = int(left_concentration)
        right_bar_length = int(right_concentration)
        
        cv2.rectangle(frame, (50, 50), (50 + left_bar_length * 2, 70), (0, 255, 0), -1)
        cv2.putText(frame, f'Left Concentration: {int(left_concentration)}%', (50, 45), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.rectangle(frame, (50, 100), (50 + right_bar_length * 2, 120), (255, 0, 0), -1)
        cv2.putText(frame, f'Right Concentration: {int(right_concentration)}%', (50, 95), 
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2)
        
        cv2.imshow("Live Pose Tracking", frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    live_tracking()
